<a href="https://colab.research.google.com/github/myatsupaing-mint/GenAI/blob/main/Fine_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.6 MB/s eta 0:00:00


In [2]:
pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
model_id = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
pip install -U transformers

In [5]:
from datasets import load_dataset
rotten = load_dataset("cornell-movie-review-data/rotten_tomatoes")
rotten

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [6]:
def compute_length(data):
  return{'length' : len(data['text'])}

rotten['train'] = rotten['train'].map(compute_length)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

In [7]:
max(rotten['train']['length'])

267

In [8]:
def tokenized(data):
  return tokenizer(data['text'], truncation = True, padding = 'max_length', max_length = 267)

trained_token = rotten['train'].map(tokenized)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

In [9]:
test_token = rotten['test'].map(tokenized)
vali_token = rotten['validation'].map(tokenized)

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [11]:
import evaluate
accuracy = evaluate.load("accuracy")

In [12]:
import numpy as np

def  compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis = 1)
  return accuracy.compute(predictions = predictions, references = labels)

In [13]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels = 2, id2label = id2label, label2id = label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
import wandb
wandb.login(key = "85f93806c875f85b6cb22dc463b260bffce079ac")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: myatsu-p65 (myatsu-p65-rangsit-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [25]:
training_args = TrainingArguments(
    output_dir = "/rotten_tomatoes",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    push_to_hub = True

)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = trained_token,
    eval_dataset = vali_token,
    processing_class = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics

)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.128900,0.534306,0.869606
2,0.071900,0.705633,0.866792


TrainOutput(global_step=1068, training_loss=0.10121945852643988, metrics={'train_runtime': 958.6989, 'train_samples_per_second': 17.795, 'train_steps_per_second': 1.114, 'total_flos': 2340773670675600.0, 'train_loss': 0.10121945852643988, 'epoch': 2.0})

In [23]:
from huggingface_hub import login
login()

In [26]:
model.push_to_hub("myatsu/bert-uncased-rotten-tomatoes-sentiment")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/myatsu/bert-uncased-rotten-tomatoes-sentiment/commit/b66444b5a1469e072a8fc03b2cacbd9de297888d', commit_message='Upload BertForSequenceClassification', commit_description='', oid='b66444b5a1469e072a8fc03b2cacbd9de297888d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/myatsu/bert-uncased-rotten-tomatoes-sentiment', endpoint='https://huggingface.co', repo_type='model', repo_id='myatsu/bert-uncased-rotten-tomatoes-sentiment'), pr_revision=None, pr_num=None)